### Get Links from a page on svWikipedia and check if the link is to a Swedish MP person   

  
* This [notebook](https://github.com/salgo60/open-data-examples/blob/master/RiksdagsvoteringGetLinks.ipynb)
  * Wikipedia [discussion related to this exercise](https://sv.wikipedia.org/wiki/Diskussion:Regeringen_Andersson#en_Wikipedia_har_med_vilka_som_r%C3%B6star_p%C3%A5_vad_borde_finnas_i_sv:Wikipedia)
  * API documentation [query](https://www.mediawiki.org/w/api.php?action=help&modules=query%2Blinks)

version 1 
* known limits
  * just get 500 links
* if more links to the same page is in the article it looks like we just get one item
  * as we have two votes --> 
     * we get 117 + 174+57 = 348 from the [first vote](https://www.riksdagen.se/globalassets/01.-aktuellt/202021/srv5-10-mpc21112410220.pdf)
     * [second vote](https://www.riksdagen.se/sv/aktuellt/2021/nov/29/magdalena-andersson-godkandes-som-statsminister/) --> 101+173+75 = 349
  * links in article to MPs will be unique members in vote one and two below we have **340** --> **I guess its wrong** in this code or in the page


TODO
* [ ] ~~get the voting data as data from [Riksdagens Öppna data](https://data.riksdagen.se/) and check that we have done our home work correct reading scanned [PDF](https://www.riksdagen.se/globalassets/01.-aktuellt/202021/srv5-10-mpc21112410220.pdf)~~ **lesson leraned not possible**
* [ ] maybe generate the Wiki syntax direct from the voting data, could maybe be used for other voting in Riksdagen that has a Wikipedia article....
* [ ] redirects


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-12-03 16:06:13.555725


In [12]:
svWikipediaAPIurl = "https://sv.wikipedia.org/w/api.php?action=query&format=json&prop=links&meta=&pllimit=500&titles=" 
pagetoCheck = "Användare:Salgo60/sandlåda"
pagetoCheck = "Statsministeromröstningen_i_Sverige_24_november_2021"

In [13]:
from IPython.core.display import display, HTML
urlcheck = svWikipediaAPIurl + pagetoCheck
urlArticle = "https://sv.wikipedia.org/wiki/" + pagetoCheck


In [14]:
display(HTML("URL sida <a href=" + urlArticle + ">" + pagetoCheck +  "</a><br>* API url <a href=" + urlcheck +">" + urlcheck + "</a>"))

In [15]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
from wikidata.client import Client


In [21]:
def getWD (url):
    print(url)
    baseUrl = "https://sv.wikipedia.org/w/api.php?action=query&prop=pageprops|wbentityusage&titles="
    title = url.replace("https://sv.wikipedia.org/wiki/","")
    urlWD = baseUrl + title
    wdr = http.request('Get', urlWD + "&format=json",
                 headers={'Content-Type': 'application/json'})

    if wdr.status != 200:
        print("Error ", wdr.status)
        return False
    
    datawd = json.loads(wdr.data.decode('utf-8'),) 
    pg_dict = datawd['query']['pages']
    pg_key = list(pg_dict.keys())[0]
    wikidata_id = pg_dict[pg_key]['pageprops']['wikibase_item']
    return wikidata_id


def getRiks(wikidata_id):
    # get SPA 
    client = Client()
    entity = client.get(wikidata_id, load=True)
    try:
        riks_prop = entity[client.get('P8388')]
    except:
        return False
    return riks_prop

In [22]:
r = http.request('GET', urlcheck,
            headers={'Content-Type': 'application/json'})
#print("http status ", r.status)
if r.status != 200:
    print("Error ",r.status)
    
data = json.loads(r.data.decode('utf-8'),)         

In [23]:
#data

In [24]:
page = data["query"]["pages"]
print("Antal länkar: \t",len(page[list(page.keys())[0]]["links"]))

Antal länkar: 	 464


In [25]:
from tqdm.notebook import trange, tqdm
import urllib.parse 

urlsv = "sv.wikipedia.org/wiki/"
urlwd = "www.wikidata.org/wiki/"
urlriks = "www.riksdagen.se/sv/ledamoter-partier/ledamot/_"
urlriksdata = "data.riksdagen.se/personlista/?utformat=json&iid="

linknr = 0
for link in tqdm(page[list(page.keys())[0]]["links"]):
    #print(link["title"])
    svName = link["title"]
    wdQnumber = getWD (svName)
    wdRiksGUID = getRiks(wdQnumber)
    if wdRiksGUID:
        linknr = linknr + 1
        svUrlArticle = "http://" + urllib.parse.quote(urlsv+svName)
        wdQnumberUrl = "http://" + urllib.parse.quote(urlwd+wdQnumber+"#P39")
        riksUrl = "http://" + urllib.parse.quote(urlriks+wdRiksGUID)
        riksData = "http://" + urlriksdata+wdRiksGUID
        display(HTML(str(linknr) + ": <a href="+svUrlArticle+">"+svName+"</a> - <a href="+wdQnumberUrl+">"+wdQnumber+"</a> " \
                + " - <a href="+riksUrl+">Riksdagen</a> " \
                + "<a href="+riksData+">json</a>"))
    else:
        print ("\t saknar Riksdagens GUID i Wikidata: \t" + svName) 
               

  0%|          | 0/464 [00:00<?, ?it/s]

26 november
	 saknar Riksdagens GUID i Wikidata: 	26 november
A-lotterierna
	 saknar Riksdagens GUID i Wikidata: 	A-lotterierna
Abraham Halef


Acko Ankarberg Johansson


Adam Marttinen


Adnan Dibrani


...

Bo Broman


Bo Toresson


Boriana Åberg


Britt Bohlin Olsson


KeyError: 'pageprops'

In [ ]:
#### Sort per party